<a href="https://colab.research.google.com/github/Shu244/CUDA-Enabled-VGG16-Replica/blob/master/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Sets up environment variables to allow cudatoolkit and numba to function properly.
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from matplotlib.image import imread
import numpy as np
from numba import cuda, vectorize
import h5py


@cuda.jit
def matrix_mul(matrix1, matrix2, result):
    '''
    Matrix 1 and 2 must be 2D arrays.
    '''

    thread_r, thread_c = cuda.grid(2)  # Gets position of thread.
    thread_r_size, thread_c_size = cuda.gridsize(2)  # Gets total number of threads in each dimension.

    result_r, result_c = result.shape

    for thread_r_i in range(thread_r, result_r, thread_r_size):
        for thread_c_i in range(thread_c, result_c, thread_c_size):
            # The following code will calculate one element in the result matrix at location thread_r_i and thread_c_i.
            for vector_i in range(0, matrix2.shape[0]):
                result[thread_r_i, thread_c_i] = result[thread_r_i, thread_c_i] + matrix1[thread_r_i, vector_i] * matrix2[vector_i, thread_c_i]


@cuda.jit
def same_convolve_multiple_filters(unpadded_input, kernal, result):
    '''
    Strides = 1.
    unpadded_input and result must have the same shape.
    Expected shape of the unpadded_input: channels, height, width
    Expected shape of the kernal: kernals, channels, height, width.
    Expected shape result: channels, height, width
    '''
    thread_channel, thread_r, thread_c = cuda.grid(3)  # Gets position of thread.
    thread_channel_size, thread_r_size, thread_c_size = cuda.gridsize(
        3)  # Gets total number of threads in each dimension.

    kernal_channels, kernal_r, kernal_c = kernal.shape[1:]

    # Padding that must be applied to all borders to maintain same dimension.
    r_pad, c_pad = ((kernal_r - 1) // 2), ((kernal_c - 1) // 2)

    result_channel, result_r, result_c = result.shape

    for thread_channel_i in range(thread_channel, result_channel, thread_channel_size):
        for thread_r_i in range(thread_r, result_r, thread_r_size):
            for thread_c_i in range(thread_c, result_c, thread_c_size):
                corner_input_r_i = thread_r_i - r_pad
                corner_input_c_i = thread_c_i - c_pad
                for kernal_channel_i in range(0, kernal_channels):
                    for kernal_r_i in range(0, kernal_r):
                        for kernal_c_i in range(0, kernal_c):
                            input_r_i = corner_input_r_i + kernal_r_i
                            input_c_i = corner_input_c_i + kernal_c_i
                            if 0 <= input_r_i < result_r and 0 <= input_c_i < result_c:
                                new_result = unpadded_input[kernal_channel_i, input_r_i, input_c_i] * kernal[thread_channel_i, kernal_channel_i, kernal_r_i, kernal_c_i]
                                result[thread_channel_i, thread_r_i, thread_c_i] = result[thread_channel_i, thread_r_i, thread_c_i] + new_result


@cuda.jit
def max_pooling_multiple_filters(input, window, result):
    '''
    Expectation: the dimensions of the input is disivible by dimensions of window. Further, windows do not overlap.
    Expected shape of the input: channels, height, width
    "window" is a tuple of format (height, width)
    Expected shape of result: channels, height, width
    '''
    thread_channel, thread_r, thread_c = cuda.grid(3)  # Gets position of thread.
    thread_channel_size, thread_r_size, thread_c_size = cuda.gridsize(3)  # Gets total number of threads in each dimension.

    window_r, window_c = window

    result_channel, result_r, result_c = result.shape

    for thread_channel_i in range(thread_channel, result_channel, thread_channel_size):
        for thread_r_i in range(thread_r, result_r, thread_r_size):
            for thread_c_i in range(thread_c, result_c, thread_c_size):
                input_r_corner_i = thread_r_i * window_r
                input_c_corner_i = thread_c_i * window_c
                max = input[thread_channel_i, input_r_corner_i, input_c_corner_i]
                for window_r_i in range(0, window_r):
                    for window_c_i in range(0, window_c):
                        element = input[thread_channel_i, input_r_corner_i + window_r_i, input_c_corner_i + window_c_i]
                        if element > max:
                            max = element
                result[thread_channel_i, thread_r_i, thread_c_i] = max


@cuda.jit
def add_biases_2D(input, biases):
    '''
    Input is of format: height, width
    There must be one bias per input element.
    is one bias per layer.
    '''
    thread_r, thread_c = cuda.grid(2)  # Gets position of thread.
    thread_r_size, thread_c_size = cuda.gridsize(2)  # Gets total number of threads in each dimension.

    result_r, result_c = input.shape

    for thread_r_i in range(thread_r, result_r, thread_r_size):
        for thread_c_i in range(thread_c, result_c, thread_c_size):
            input[thread_r_i, thread_c_i] = input[thread_r_i, thread_c_i] + biases[thread_r_i, thread_c_i]


@cuda.jit
def add_biases_3D(input, biases):
    '''
    Input is of format: channels, height, width
    The number of biases must be equal to the number of layers as there
    is one bias per layer.
    '''
    thread_channel, thread_r, thread_c = cuda.grid(3)  # Gets position of thread.
    thread_channel_size, thread_r_size, thread_c_size = cuda.gridsize(3)  # Gets total number of threads in each dimension.

    result_channel, result_r, result_c = input.shape

    for thread_channel_i in range(thread_channel, result_channel, thread_channel_size):
        for thread_r_i in range(thread_r, result_r, thread_r_size):
            for thread_c_i in range(thread_c, result_c, thread_c_size):
                input[thread_channel_i, thread_r_i, thread_c_i] = input[thread_channel_i, thread_r_i, thread_c_i] + biases[thread_channel_i]


# Using float64 because h5 file stores parameters in float64.
@vectorize(['float32(float32)', 'float64(float64)'], target='cuda')
def ReLU(activation):
    if activation > 0:
        return activation
    return 0
    ''' 
    return max(activation, 0) causes racing condition to occur. 
    '''


def softmax(activations):
    '''
    Used only in last layer with 1000 neurons. Not worth the data transfer speed
    to involve GPU.
    '''
    eactivations = np.exp(activations)
    etotal = sum(eactivations)
    return eactivations / etotal


class VGG16:
    def __init__(self, params_path):
        # reading in weights and baises now.
        self.params = h5py.File(params_path, 'r')
        self.input = None

    def classify(self, image_path):
        img = imread(image_path)
        # Flips image from RGB to BGR
        img = np.flip(img, [2])
        # Gets image to be in the format: channels, height, width
        img = np.rollaxis(img, 2, 0)  
        print('Tranposed original image:\n\n',img[0,:5,:5])
        # Put img array in C-contiguous format
        img = img.copy(order='C')
        img_device = cuda.to_device(img)
        self.input = img_device
        return self.forward_propagation()

    def forward_propagation(self):
        layers_with_pool = (1, 3, 6, 9, 12)
        threads_per_block = (4, 8, 16)
        blocks_per_grid = (10, 10)
        pool_window_height, pool_window_width = 2, 2
        
#         #TEST
#         orinput = self.input.copy_to_host()
#         print('original image:\n\n',orinput[0,:5,:5])

        # going through all convolution layers.
        for i in range(0, 13):
            filters = self.params[str(i)]['weights']    
            # Eventually remove this step by saving the data properly. This flips weights so it is in proper format
            filters = np.array(filters)
            filters= filters[:,:,::-1,::-1]
            filters = np.ascontiguousarray(filters)      
            biases = self.params[str(i)]['biases']
            filters_device = cuda.to_device(filters)
            biases_device = cuda.to_device(biases)

            # Computes convolution.
            result = np.zeros((filters.shape[0],) + self.input.shape[1:])
            out_device = cuda.device_array_like(result)
            same_convolve_multiple_filters[blocks_per_grid, threads_per_block](self.input, filters_device, out_device)

            # Applying biases now.
            add_biases_3D[blocks_per_grid, threads_per_block](out_device, biases_device)          

            # Applying ReLU now.
            out_device = ReLU(out_device)           
            
#             # Test print
#             test = out_device.copy_to_host()
#             print('weights:\n\n', filters[0,2,:,:])
#             print("outputs: \n\n", test[0, :10, :10])
#             return

            # Applying maxpool now, if necessary.
            if i in layers_with_pool:
                channels, height, width = out_device.shape
                window_device = cuda.to_device((pool_window_height, pool_window_width))
                new_out_device = cuda.device_array((channels, height // pool_window_height, width // pool_window_width))
                max_pooling_multiple_filters[blocks_per_grid, threads_per_block](out_device, window_device, new_out_device)
                out_device = new_out_device

            self.input = out_device

        # Must apply fully connected layers now.
        threads_per_block = (32, 16)
        blocks_per_grid = (10, 10)
        # Converting volume to vector.
        num_elements = np.prod(self.input.shape)
        self.input = self.input.reshape((1, num_elements))
        for i in range(13, 16):
            # Setting up for CUDA Numba.
            weights = self.params[str(i)]['weights']
            biases = self.params[str(i)]['biases']
            weights_device = cuda.to_device(weights)
            biases_device = cuda.to_device(biases)

            mult_result = np.zeros((self.input.shape[0], weights.shape[1]))
            device_mult_result = cuda.to_device(mult_result)

            # Performing matrix multiplication on GPU.
            matrix_mul[blocks_per_grid, threads_per_block](self.input, weights_device, device_mult_result)

            # Must add biases now.
            biases_device = biases_device.reshape(device_mult_result.shape)
            add_biases_2D(device_mult_result, biases_device)          

            # Must apply ReLU now
            self.input = ReLU(device_mult_result)

        # Now applying softmax functon.
        result = self.input.copy_to_host()
        result = result.reshape(result.shape[1])
        return softmax(result)
    
    def save_weights(path):
        hf = h5py.File(path, 'w')
        hf.close()
    
    def run_one_epoch(dataset, batch_size):
        start = 0
        end = batch_size
        L = len(dataset)
        while start <= L:     
            mini_batch = dataset[start:end]
            
            for one_data in mini_batch:
                image_path, label = one_data
                logits = self.classifies(image_path)
                
            
            start = start + batch_size
            end = end + bathc_size
    
    def train(dataset, epochs, batch_size):
        '''
        data set is array of tuple = (feature, label)
        '''
        dataset = np.random.shuffle(dataset)
        

In [0]:
image_path1 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/dog photo.jpg'
image_path2 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/orangutan2.jpg'
image_path3 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/orangutan3.jpg'
image_path4 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/orangutan4.jpg'
image_path5 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/orangutan5.jpg'
image_path6 = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/orangutan6.jpg'
data_path = '/content/drive/My Drive/Colab Notebooks/VGG16 Test Data/vgg16_weights_reformatted.h5'

# #Test
# from PIL import Image
# img = Image.open(image_path1)
# img = np.flip(img, [2])
# img = np.expand_dims(img, axis=0)
# print("Just read Image:\n\n", img[0,:5,:5,0])

model = VGG16(data_path)
print(np.argmax(model.classify(image_path1)))
# print(model.classify(image_path2).argmax())
# print(model.classify(image_path3).argmax())
# print(model.classify(image_path4).argmax())
# print(model.classify(image_path5).argmax())
# print(model.classify(image_path6).argmax())



Tranposed original image:

 [[145 145 146 146 147]
 [144 145 145 146 147]
 [144 144 144 145 146]
 [143 143 144 144 145]
 [141 141 142 142 143]]
original image:

 [[145 145 146 146 147]
 [144 145 145 146 147]
 [144 144 144 145 146]
 [143 143 144 144 145]
 [141 141 142 142 143]]
weights:

 [[ 0.4800154   0.4085474  -0.06514555]
 [ 0.31047726  0.05020237 -0.40338343]
 [-0.05087169 -0.2852275  -0.41851634]]
outputs: 

 [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          3.67474568  3.35007679  1.37565672  1.15158713  2.49530613
   2.45618212  6.6246382   6.22197354  2.53823292]
 [ 0.          4.56247675  3.71220553  2.21119511  2.59774458  3.86510336
   3.37345707  5.51669276  4.92909062  2.53823292]
 [ 0.          4.70175135  3.74522173  3.04567969  2.29337418  3.73705447
   3.65078461  3.7896663   4.08028138  2.45343316]
 [ 0.          4.49325573  4.96670783  4.50492871  2.29274046  2.51233923
   3.77968132  

In [0]:
a = np.array([[1,2,3],[4,5,6],[7,8,9],[11,12,13]])
b = np.flip(a, (1))
print(b)

[[ 3  2  1]
 [ 6  5  4]
 [ 9  8  7]
 [13 12 11]]


In [0]:
def forward_propagation(image_path):
    img = imread(image_path)
    # Gets image to be in the format: channels, height, width
    img = np.transpose(img, (2, 1, 0))
    # Put img array in C-contiguous format
    img = img.copy(order='C')
    img_device = cuda.to_device(img)
    input = img_device
    
    layers_with_pool = (1, 3, 6, 9, 12)
    threads_per_block = (4, 8, 16)
    blocks_per_grid = (10, 10)
    pool_window_height, pool_window_width = 2, 2
    
    # layers, num filters, channels, height, width
    all_filters = np.array([
        [
            [
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.3, 0.1],
                    [-0.2, -0.1, -0.1]
                ],
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ]
            ],
            [             
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.3, 0.1],
                    [-0.2, -0.1, -0.1]
                ],
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ]
            ],
            [
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],                
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.3, 0.1],
                    [-0.2, -0.1, -0.1]
                ]
            ]
        ],
        [
            [
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.5, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.3, -0.1, 0.2],
                    [0.3, 0.4, 0.1],
                    [-0.2, -0.1, -0.1]
                ]
            ],
            [                
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.5, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.3, -0.1, 0.2],
                    [0.3, 0.4, 0.1],
                    [-0.2, -0.1, -0.1]
                ]
            ],
            [               
                [
                    [0.1, -0.1, 0.2],
                    [0.3, 0.2, 0.1],
                    [-0.1, -0.1, -0.1]
                ],
                [
                    [0.3, -0.1, 0.2],
                    [0.3, 0.4, 0.1],
                    [-0.2, -0.1, -0.1]
                ],
                [
                    [0.2, -0.1, 0.2],
                    [0.1, 0.5, 0.1],
                    [-0.1, -0.1, -0.1]
                ]
            ]
        ]
    ])
    all_biases = np.array([
        [
            1, 2, 3
        ],
        [
            4, 5, 6
        ]
    ])

    # going through all convolution layers.
    for i in range(0, 13):
        filters = all_filters[i]
        biases = all_biases[i]
        filters_device = cuda.to_device(filters)
        biases_device = cuda.to_device(biases)

        # Printing out sample of inputs for debugging
        test_input = input.copy_to_host()
        print('Layer ' + str(i) + ' input:\n\n', test_input[:, :4, :4], '\n\n')
        print('weights: \n\n', filters[:, :3, :4, :4], '\n\n')
        print('biases: \n\n', biases[:3], '\n\n')

        # Computes convolution.
        result = np.zeros((filters.shape[0],) + input.shape[1:])
        out_device = cuda.device_array_like(result)
        same_convolve_multiple_filters[blocks_per_grid, threads_per_block](input, filters_device, out_device)

        out_device_test = out_device.copy_to_host()
        print('output:\n\n', out_device_test[:, :3, :3], '\n\n')
        
        # Applying biases now.
        add_biases_3D[blocks_per_grid, threads_per_block](out_device, biases_device)

        # Applying ReLU now.
        out_device = ReLU(out_device)

        # Applying maxpool now, if necessary.
        if i in layers_with_pool:
            # Printing out input to maxpool 
            test_test = out_device.copy_to_host()
            print('input for maxpool: \n\n', test_test[:, :2, :2])
            
            channels, height, width = out_device.shape
            window_device = cuda.to_device((pool_window_height, pool_window_width))
            new_out_device = cuda.device_array((channels, height // pool_window_height, width // pool_window_width))
            max_pooling_multiple_filters[blocks_per_grid, threads_per_block](out_device, window_device, new_out_device)
            out_device = new_out_device

            # Printing out sample of inputs for debugging
            test_out = out_device.copy_to_host()
            print('Maxpool: \n\n', test_out[:, 0, 0], '\n\n')
            return None

        input = out_device



In [0]:
forward_propagation(image_path1)

In [0]:
import numpy as np
from numba import cuda, vectorize
import scipy.signal as sps

input = np.array(
[
 [[124, 148, 167, 174],
  [145, 148, 147, 162],
  [125, 148, 169, 185],
  [131, 163, 185, 191]
 ],

 [[157, 181, 199, 203],
  [177, 177, 176, 189],
  [151, 174, 192, 208],
  [150, 182, 204, 210]
 ],

 [[110, 134, 152, 159],
  [128, 129, 130, 144],
  [104, 127, 146, 162],
  [105, 137, 159, 165]
 ]
] )

# layers, num filters, channels, height, width
all_filters = np.array([
    [
        [
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.3, 0.1],
                [-0.2, -0.1, -0.1]
            ],
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ]
        ],
        [
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.3, 0.1],
                [-0.2, -0.1, -0.1]
            ],
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ]
        ],
        [
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.3, 0.1],
                [-0.2, -0.1, -0.1]
            ]
        ]
    ],
    [
        [
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.5, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.3, -0.1, 0.2],
                [0.3, 0.4, 0.1],
                [-0.2, -0.1, -0.1]
            ]
        ],
        [
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.5, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.3, -0.1, 0.2],
                [0.3, 0.4, 0.1],
                [-0.2, -0.1, -0.1]
            ]
        ],
        [
            [
                [0.1, -0.1, 0.2],
                [0.3, 0.2, 0.1],
                [-0.1, -0.1, -0.1]
            ],
            [
                [0.3, -0.1, 0.2],
                [0.3, 0.4, 0.1],
                [-0.2, -0.1, -0.1]
            ],
            [
                [0.2, -0.1, 0.2],
                [0.1, 0.5, 0.1],
                [-0.1, -0.1, -0.1]
            ]
        ]
    ]
])
all_biases = np.array([
    [
        1, 2, 3
    ],
    [
        4, 5, 6
    ]
])

channel = 2
filters = all_filters[0, 2, channel, :, :]
cor_input = np.pad(input[channel], (1, 1), mode='constant')[:5, :5]


a = sps.correlate(cor_input, filters, mode='valid')

print(a)

[[20.7 14.9 21.7]
 [44.  55.4 53.5]
 [32.7 51.2 54.5]]
